In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from lib.fcn import FCN
from lib.attack import IterativeFGSMAttack

In [9]:
gpu_name = "cuda:0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
path = "model/FCN_MNIST"
model = FCN()
model.load_state_dict(torch.load(path, map_location=gpu_name))
model = model.to(device)

In [11]:
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
])
_test = torchvision.datasets.MNIST(root='data/mnist', train=False,
                                   download=True, transform=transform)
test = torch.utils.data.DataLoader(_test, batch_size=32,
                                   shuffle=False, num_workers=2)

In [12]:
total = 0
correct = 0
adv_correct = 0

fgsm = IterativeFGSMAttack(model, criterion=torch.nn.CrossEntropyLoss(),
                           num_steps=0.01,
                           step_size=40,
                           epsilon=0.3)
        
for data in test:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    adv_inputs = fgsm.generate(inputs)
            
    outputs = model(inputs)
    adv_outputs = model(adv_inputs)
            
    prediction = outputs.max(1)[1]
    adv_prediction = adv_outputs.max(1)[1]
            
    total += labels.size(0)
    correct += prediction.eq(labels).sum().item()
    adv_correct += adv_prediction.eq(labels).sum().item()

    test_acc = 100*correct/total
    adv_acc = 100*adv_correct/total

print("Test: %.2f%%     Adv: %.2f%%"%(test_acc, adv_acc))

In [13]:
inputs = np.squeeze(inputs.detach().cpu().numpy())
adv_inputs = np.squeeze(adv_inputs.detach().cpu().numpy())

In [14]:
idx = 0
f, axs = plt.subplots(1,2,figsize=(10,4))
axs[0].set_xlabel('pure')
axs[0].imshow(inputs[idx])
axs[1].set_xlabel('adv')
axs[1].imshow(adv_inputs[idx])